#   *Carregando as bibliotecas*

In [ ]:
import pandas as pd
import numpy as np
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# Disable warnings
import warnings
warnings.filterwarnings("ignore")

# Import plotting modules
!pip install chart_studio
import seaborn as sns
sns.set()
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker
import plotly.express as px
from plotly.offline import iplot

import chart_studio.plotly as py
import plotly.graph_objs as go
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')
%matplotlib inline

warnings.filterwarnings("ignore")
import plotly.figure_factory as ff
from colorama import Fore, Back, Style 

# Import machine learning algorithms
import xgboost
import lightgbm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
sns.set_context(
    "notebook", 
    font_scale=1.5,       
    rc={ 
        "figure.figsize": (11, 8), 
        "axes.titlesize": 18 
    }
)

from matplotlib import rcParams
rcParams['figure.figsize'] = 11, 8

**Load dataset**

In [ ]:
df = pd.read_csv("../input/mlcourse/mlbootcamp5_train.csv")
print("dataset size: ", df.shape)
df.head(10)

**Drop column id**

In [ ]:
dff = df.drop("id", axis=1)

In [ ]:
dff.head(5)

In [ ]:
dff['gluc']

# Contando e agrupando as features 

In [ ]:
df_uniques = pd.melt(frame=dff, value_vars=['gender','cholesterol', 
                                           'gluc', 'smoke', 'alco', 
                                           'active', 'cardio'])
df_uniques = pd.DataFrame(df_uniques.groupby(['variable', 
                                              'value'])['value'].count()) \
    .sort_index(level=[0, 1]) \
    .rename(columns={'value': 'count'}) \
    .reset_index()

sns.factorplot(x='variable', y='count', hue='value', 
               data=df_uniques, kind='bar', size=12);

**Com base nos gráfics acima, as features  apresentam um considerável desbalanceamento. Vemos que a maioria dos indivíduos praticam atividades física, não consomem álcool regularmente, apresentam níveis normais de glicóse e os não fumantes também são prevalecentes. Contudo, a quantidade de cardios e não cardios são bem balanceados.**

In [ ]:
df_uniques = pd.melt(frame=dff, value_vars=['gender','cholesterol', 
                                           'gluc', 'smoke', 'alco', 
                                           'active'], 
                     id_vars=['cardio'])
df_uniques = pd.DataFrame(df_uniques.groupby(['variable', 'value', 
                                              'cardio'])['value'].count()) \
    .sort_index(level=[0, 1]) \
    .rename(columns={'value': 'count'}) \
    .reset_index()

sns.factorplot(x='variable', y='count', hue='value', 
               col='cardio', data=df_uniques, kind='bar', size=9);

**No gráfico acima ao compararmos as features dos cardios e não cardios, vemos que a quantidade de indivíduos com níveis mais elevados de colesterol, glicose e sendentárismo são maiores do que a quantidade de indivíduos não cardios, sendo estes os fatores mais relevantes. Surpreendetemente, não há muita difença entre os níveis de consulmo de álcool e fumantes quando comparamos os indíduos cardios com os não cardios. Isso pode estar relacinado com baixa quantidade de índividos com essas características quando comparamos a quantidade total de indíduos.**

# Others factors hist plots

In [ ]:
dff['age'].iplot(kind='hist',
                              xTitle='Idade(Dias)', 
                              yTitle='Quantidade',
                              linecolor='black', 
                              opacity=0.7,
                              color='#FB8072',
                              theme='pearl',
                              bargap=0.2,
                              gridcolor='white',
                              title='Distribuição das idades')

In [ ]:
fig = px.histogram(dff, x="age", color="cardio", marginal="violin", hover_data=dff.columns)
fig.show()

Nesse gráfico vemos a distribuição da quantidade de índividos  não cardios e cardios com relação a idade. Como é de se esperar, a quantidade de indívidos cardios cresce a medida que a quantidade invíduos saudávies vai diminuindo com o avançado da idade. 

In [ ]:
fig = px.scatter(dff, x="weight", y="age", color='cardio')
fig.show()

**Fica claro que existe um relação direta entre idade e sobrepeso com relação aos índivíduos com algum problema cardíaco.**

In [ ]:
fig = px.scatter(dff, x="height", y="age", color='cardio')
fig.show()

**A distribuição de idades com as alturas, como mostra o gráfico, são relativamente homogêneas. Parecendo que há uma região com ínvíduos com idade entre 14mil e 16mil dias (~38 anos e ~44 anos) com alturas aproximadas entre 1.45m e 1.74m.**

In [ ]:
fig = px.scatter(dff, x="ap_hi", y="age", color='cardio')
fig.show()

In [ ]:
corrmat = dff.corr() 
f, ax = plt.subplots(figsize =(12, 10)) 
sns.heatmap(corrmat, ax = ax, cmap = 'RdYlBu_r', linewidths = 0.5) 

# Aplicando modelos de predição.

**Usaremos modelos de machine larning para prevermos qual a probalidade de um indivídou com base em suas features venha a ser tornar cardio ou não.**

In [ ]:
# Separando as labels e variáveis alvo.
y = dff["cardio"]
x = dff.drop("cardio", axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)


In [ ]:
accuracy_list = []

In [ ]:
# Decision Tree Classifier

dt_clf = DecisionTreeClassifier(max_leaf_nodes=10, random_state=30, criterion='entropy')
dt_clf.fit(x_train, y_train)
dt_pred = dt_clf.predict(x_test)
dt_acc = dt_clf.score(x_test,y_test)
accuracy_list.append(100*dt_acc)

In [ ]:
print(Fore.GREEN + "Accuracy of Decision Tree Classifier is : ", "{:.2f}%".format(100* dt_acc))

In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, dt_pred)
sns.heatmap(mat.T, square=True, annot=True,fmt="d", cbar = False)
plt.title("Tree Clasifier - Confusion Matrix")
plt.xticks(range(2), ["Heart Not Disease","Heart Disease"], fontsize=16)
plt.yticks(range(2), ["Heart Not Disease","Heart Disease"], fontsize=16)
plt.xlabel("true label")
plt.ylabel("predicted label");

In [ ]:
# K Neighbors Classifier

kn_clf = KNeighborsClassifier(n_neighbors=6)
kn_clf.fit(x_train, y_train)
kn_pred = kn_clf.predict(x_test)
kn_acc = kn_clf.score(x_test,y_test)
accuracy_list.append(100*kn_acc)


In [ ]:
print(Fore.GREEN + "Accuracy of K Neighbors Classifier is : ", "{:.2f}%".format(100* kn_acc))

In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, kn_pred)
sns.heatmap(mat.T, square=True, annot=True,fmt="d", cbar = False)
plt.xlabel("true label")
plt.ylabel("predicted label")
plt.title("K Neighbors Classifier - Confusion Matrix")
plt.xticks(range(2), ["Heart Not Disease","Heart Disease"], fontsize=16)
plt.yticks(range(2), ["Heart Not Disease","Heart Disease"], fontsize=16);

In [ ]:
# RandomForestClassifier
r_clf = RandomForestClassifier(max_features=0.5, max_depth=15, random_state=1)
r_clf.fit(x_train, y_train)
r_pred = r_clf.predict(x_test)
r_acc = r_clf.score(x_test,y_test)
accuracy_list.append(100*r_acc)

In [ ]:
print(Fore.GREEN + "Accuracy of Random Forest Classifier is : ", "{:.2f}%".format(100* r_acc))

In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, r_pred)
sns.heatmap(mat.T, square=True, annot=True,fmt="d", cbar = False)
plt.xlabel("true label")
plt.ylabel("predicted label")
plt.title("Random Forest Classifier - Confusion Matrix")
plt.xticks(range(2), ["Heart Not Disease","Heart Disease"], fontsize=16)
plt.yticks(range(2), ["Heart Not Disease","Heart Disease"], fontsize=16);

In [ ]:
# GradientBoostingClassifier

gradientboost_clf = GradientBoostingClassifier(max_depth=2, random_state=4)
gradientboost_clf.fit(x_train,y_train)
gradientboost_pred = gradientboost_clf.predict(x_test)
gradientboost_acc = gradientboost_clf.score(x_test,y_test)
accuracy_list.append(100*gradientboost_acc)

In [ ]:
print(Fore.GREEN + "Accuracy of Gradient Boosting is : ", "{:.2f}%".format(100* gradientboost_acc))

In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, gradientboost_pred)
sns.heatmap(mat.T, square=True, annot=True,fmt="d", cbar = False)
plt.xlabel("true label")
plt.ylabel("predicted label")
plt.title("Random Forest Classifier - Confusion Matrix")
plt.xticks(range(2), ["Heart Not Disease","Heart Disease"], fontsize=16)
plt.yticks(range(2), ["Heart Not Disease","Heart Disease"], fontsize=16);

In [ ]:
model_list = ['DecisionTree', 'KNearestNeighbours', 'RandomForest', 'GradientBooster']

In [ ]:
plt.rcParams['figure.figsize']=20,8
sns.set_style('darkgrid')
ax = sns.barplot(x=model_list, y=accuracy_list, palette = "vlag", saturation =2.0)
plt.xlabel('Classifier Models', fontsize = 20 )
plt.ylabel('% of Accuracy', fontsize = 20)
plt.title('Accuracy of different Classifier Models', fontsize = 20)
plt.xticks(fontsize = 12, horizontalalignment = 'center', rotation = 8)
plt.yticks(fontsize = 12)
for i in ax.patches:
    width, height = i.get_width(), i.get_height()
    x, y = i.get_xy() 
    ax.annotate(f'{round(height,2)}%', (x + width/2, y + height*1.02), ha='center', fontsize = 'x-large')
plt.show()

**Testamos quatro modelos de machine learning, Decision Tree, KNearest Neighbours, Random Forest e Gradient Booster. Como compilado no gráfico acima, o Gradient Booster se provou ser o melhor modelo de previsão para os nossos dados. Embora, com exceção do modelo KNearest Neighbour todos os outros apresentam acuracia semelhante.**

In [ ]:
sns.boxplot( y= dff['age'] )